In [ ]:
# 预测后的数据的清洗

In [2]:
import pandas as pd
import numpy as np

In [65]:
predict_file = pd.read_csv("./datasets/deepseek_0.csv", on_bad_lines='skip')
# predict_file = pd.read_csv("predict_file_copy.csv")

In [ ]:
print(predict_file)

In [12]:
# 看一下从哪里开始缺失的
for i in range(predict_file.shape[0]):
    if i != int(predict_file.iloc[i].iloc[0]):
        print(i)
        break

In [ ]:
# 计算有多少个数据的预测概率相加不为一
l1 = 0

# 计算有多少个数据 预测概率缺失
l2 = 0

for i in range(predict_file.shape[0]):
    attack = predict_file.at[i, "attack"]
    benign = predict_file.at[i, "benign"]
    # print(i)
    if pd.isna(attack) or pd.isna(benign):
        l2 += 1
    else: 
        if float(attack) + float(benign) != 1:
            l1 += 1      

print(l1, l2)


In [6]:
# 验证float转化
for i in range(predict_file.shape[0]):
    float(predict_file.at[i, 'attack'])

In [ ]:
print(predict_file)

In [66]:
ori_file = pd.read_csv("./datasets/original.csv")

labels = []
pre_result = []


for i in range(predict_file.shape[0]):
    attack = predict_file.at[i, "attack"]
    benign = predict_file.at[i, "benign"]

    if pd.isna(attack) or pd.isna(benign):
        pass
    else:
        if ori_file.at[i, " Label"] != 'BENIGN':
            labels.append(0)
        else:
            labels.append(1)

        probabilities = np.array([attack, benign], dtype=float)
        normalized_probabilities = probabilities / np.sum(probabilities)
        
        if np.sum(probabilities) == 0:
            print(i)
        
        pre_result.append(normalized_probabilities)



In [ ]:
print(len(labels))
print(len(pre_result))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score, f1_score

# labels = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
# pre_result = [
#     [0.650, 0.350],
#     [0.100,  0.900],
#     [0.85, 0.15],
#     [0.950, 0.050],
#     [0.900, 0.100],
#     [0.75, 0.25],
#     [0.800, 0.200],
#     [0.10, 0.90],
#     [0.85, 0.15],
#     [0.1, 0.9],
#     [0.780, 0.220],
#     [0.85, 0.15],
# ]

# Convert prediction results to labels (using the class with the highest probability)
predicted_labels = np.argmax(pre_result, axis=1)

# Calculate metrics
precision = precision_score(labels, predicted_labels)
recall = recall_score(labels, predicted_labels)
f1 = f1_score(labels, predicted_labels)
positive_probabilities = [prob[1] for prob in pre_result]
auc = roc_auc_score(labels, positive_probabilities)

# Print the metrics
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"AUC: {auc}")

# Plot confusion matrix
conf_matrix = confusion_matrix(labels, predicted_labels)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

# Provided data
# labels = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
# pre_result = [
#     [0.650, 0.350],
#     [0.100,  0.900],
#     [0.85, 0.15],
#     [0.950, 0.050],
#     [0.900, 0.100],
#     [0.75, 0.25],
#     [0.800, 0.200],
#     [0.10, 0.90],
#     [0.85, 0.15],
#     [0.1, 0.9],
#     [0.780, 0.220],
#     [0.85, 0.15],
# ]

# Convert prediction results to labels (using the class with the highest probability)
predicted_labels = np.argmax(pre_result, axis=1)

# Extract the probabilities for the positive class
positive_probabilities = [prob[1] for prob in pre_result]

# Calculate AUC
auc = roc_auc_score(labels, positive_probabilities)

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(labels, positive_probabilities)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC = {auc:.4f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()
